In [48]:
import pandas as pd
import math

In [49]:
csv = "data/new_data.csv"
df = pd.read_csv(csv)
df = df.drop_duplicates()


In [50]:
# Функция для вычисления расстояния между координатами
def get_distance(lat1, long1, lat2, long2):
    R = 6373
    lat1 = math.radians(lat1)
    long1 = math.radians(long1)
    lat2 = math.radians(lat2)
    long2 = math.radians(long2)
    dlong = long2 - long1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlong / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c
    

In [51]:
# Функция для вычисления длины всего маршрута
def get_route_distance(id, df):
    distance = 0
    rows = df.loc[df['id'] == id]
    for i in range(1, len(rows)):
        distance += get_distance(rows['latitude'].iloc[i-1], rows['longitude'].iloc[i-1], rows['latitude'].iloc[i], rows['longitude'].iloc[i])
    return distance

In [52]:
alt_inaccuracy = 50 # погрешность в изменении высоты
# Функция для получения доли участков, где самолёт менял высоту
def get_altitude_rate(id, df):
    rows = df.loc[df['id'] == id]
    point_amount = len(rows)
    if point_amount <= 1:
        return 1.
    id_distance = get_route_distance(id, df)
    if id_distance == 0:
        return 1.
    altitudes = rows['altitude']
    latitudes = rows['latitude']
    longitudes = rows['longitude']
    alt_delta_ratio = 0
    for i in range(1, point_amount):
        if abs(altitudes.iloc[i] - altitudes.iloc[i-1]) < alt_inaccuracy:
            alt_delta_ratio += get_distance(latitudes.iloc[i-1], longitudes.iloc[i-1], latitudes.iloc[i], longitudes.iloc[i])
    return alt_delta_ratio / id_distance

In [53]:
heading_inaccuracy = 2 # погрешность в изменении направления
# Функция для получения доли участков, где самолёт менял направление
def get_heading_rate(id, df):
    rows = df.loc[df['id'] == id]
    point_amount = len(rows)
    if point_amount <= 1:
        return 1.
    id_distance = get_route_distance(id, df)
    if id_distance == 0:
        return 1
    headings = rows['heading']
    latitudes = rows['latitude']
    longitudes = rows['longitude']
    head_delta_ratio = 0
    for i in range(1, point_amount):
        if abs(headings.iloc[i] - headings.iloc[i-1]) < heading_inaccuracy:
            head_delta_ratio += get_distance(latitudes.iloc[i-1], longitudes.iloc[i-1], latitudes.iloc[i], longitudes.iloc[i])
    return head_delta_ratio / id_distance

In [54]:
# Функция для получения доли участков, где самолёт менял высоту и направление
def get_heading_altitude_rate(id, df):
    return get_heading_rate(id, df) * get_altitude_rate(id, df)

In [55]:
ids = {id: df['id'].unique() for id in df['id']}
altitude_rates = []
heading_rates = []
heading_altitude_rates = []

# создаём новый датафрейм, группируя информацию по id полётов

for id in ids:
    altitude_rates.append(get_altitude_rate(id, df))
    heading_rates.append(get_heading_rate(id, df))
    heading_altitude_rates.append(get_heading_altitude_rate(id, df))
print(altitude_rates)
print(heading_rates)
print(heading_altitude_rates)

df = pd.DataFrame(list(zip(ids, altitude_rates, heading_rates, heading_altitude_rates)),
    columns = ['ids', 'altitude_rates', 'heading_rates', 'heading_altitude_rates'])
df = df[df['heading_altitude_rates'] != 0]
display(df)


[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3394508234037636, 1.0, 1.0, 0.3580673975385481, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6285774474331718, 1.0, 1.0, 1.0, 0.4252098100825923, 1.0, 1.0, 0.6620607633108107, 0.846809930254448, 1.0, 0.5323843523230174, 0.2681023078273304, 1.0, 0.5716249957038326, 0.5708182860859654, 1.0, 1.0, 1.0, 1.0, 1.0, 0.115826331803759, 0.6366164413032641, 1.0, 1.0, 1.0, 1.0, 0.5331734335888625, 0.8653676923389135, 1.0, 1.0, 1.0, 0.560286895040117, 0.8558918902294699, 0.750778707708127, 0.6778211104637137, 0.0033199594865155053, 1.0, 1.0, 0.9557565953680709, 1.0, 1.0, 1.0, 0.6866855932702853, 1.0, 0.06092197965643796, 0.2109725190944443, 1.0, 1.0, 0.01998393672052629, 1.0, 1.0, 0.6752055478349276, 1.0, 0.3353149332008208, 0.9686338227218798, 1.0, 0.44148882427030234, 0.2086872546723977, 1.0, 0.8174635486324828, 0.6627841343837048, 0.9260117167210031, 0.8586509325630504, 0.04193236438650496, 1.0, 1.0, 0.46828436702286697, 1.0, 1.0, 1.0, 0.24801112385493318, 0.5094758

,ids,altitude_rates,heading_rates,heading_altitude_rates
0,327b1463,1.0,1.000000,1.000000
1,327b1959,1.0,1.000000,1.000000
2,327b0897,1.0,1.000000,1.000000
3,327be891,1.0,1.000000,1.000000
4,327c4690,1.0,1.000000,1.000000
...,...,...,...,...
589,327f566f,1.0,0.450887,0.450887
590,327fc194,1.0,1.000000,1.000000
591,327f9076,1.0,1.000000,1.000000
592,327f832d,1.0,1.000000,1.000000


In [56]:
print("Описание доли маршрута полёта без изменения высоты")
print(df['altitude_rates'].describe())
print("Описание доли маршрута полёта без поворота")
print(df['heading_rates'].describe())
print("Описание доли маршрута полёта без поворота и без снижения высоты")
print(df['heading_altitude_rates'].describe())


Описание доли маршрута полёта без изменения высоты
count    562.000000
mean       0.785184
std        0.291153
min        0.003320
25%        0.537503
50%        1.000000
75%        1.000000
max        1.000000
Name: altitude_rates, dtype: float64
Описание доли маршрута полёта без поворота
count    562.000000
mean       0.757657
std        0.268399
min        0.015244
25%        0.613913
50%        0.845550
75%        1.000000
max        1.000000
Name: heading_rates, dtype: float64
Описание доли маршрута полёта без поворота и без снижения высоты
count    562.000000
mean       0.619119
std        0.341057
min        0.000306
25%        0.303569
50%        0.660685
75%        1.000000
max        1.000000
Name: heading_altitude_rates, dtype: float64
